In [1]:
import pandas as pd
import seaborn as sns
import os

In [3]:
pd.set_option("display.max_columns", None)

In [2]:
print(os.getcwd())

/Users/matheus/Projects/steam_analysis/steam-analysis/notebooks/analysis


In [3]:
os.chdir("../..")

In [4]:
steam_df = pd.read_csv("data/interim/steam-transformed-data.csv")

In [26]:
steam_df.groupby(by="year").count()

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,success_1000_threshold,success_500_threshold
year,,,,,,,,,,,,,,,,,,,,,
1997,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2002,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2003,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2004,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2005,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2006,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
2007,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
2008,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
2009,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85


In [22]:
steam_df[["total_reviews", "success_1000_threshold"]].groupby(
    by="success_1000_threshold"
).mean()

,total_reviews
success_1000_threshold,
0,62.695755
1,17218.117707


In [27]:
numeric_columns = steam_df.select_dtypes(include=[float, int, bool]).columns

In [43]:
steam_df_2023 = steam_df[steam_df["year"] >= 2023]

In [44]:
steam_df_2023[numeric_columns].groupby(by="success_1000_threshold").median()

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_500_threshold
success_1000_threshold,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,4.79,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1045.0,7.0,1.0,1.0,1.0,0.0,0.000000,1.0,0.0,1.0,0.0,0.0,7.0,16.0,2023.0,0.0
1,2278.0,0.0,24.99,1.0,1.0,0.0,0.0,0.0,0.0,1859.0,360.0,NaN,38.0,2068.5,354.5,134.5,356.5,157.5,1716.0,10.0,2.0,20.0,11.0,1.0,5.646919,0.0,2421.0,1.0,1.0,1.0,9.0,16.0,2023.0,1.0


## Limiting the dataset

In [56]:
steam_df_indie = steam_df_2023[
    steam_df_2023["Genres"].str.contains("Indie").fillna(False)
].sort_values(by="total_reviews", ascending=False)
steam_df_indie = steam_df_indie[steam_df_indie["Metacritic score"] == 0]
steam_df_indie = steam_df_indie[steam_df_indie["total_reviews"] > 10]

In [57]:
steam_df_indie = steam_df_indie[steam_df_indie["total_reviews"] > 10]

In [59]:
steam_df_indie[numeric_columns].groupby(by="success_500_threshold").median()

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold
success_500_threshold,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2.0,0.0,8.99,0.0,1.0,0.0,0.0,0.0,0.0,25.0,3.0,NaN,12.0,0.0,0.0,0.0,0.0,0.0,1334.0,8.0,1.0,20.0,2.0,0.0,7.500000,0.0,29.0,1.0,0.0,0.0,8.0,16.0,2023.0,0.0
1,432.0,0.0,17.99,0.0,1.0,0.0,0.0,0.0,0.0,847.5,130.5,NaN,28.0,957.0,115.0,2.5,127.5,2.5,1695.5,10.0,2.0,20.0,7.0,1.0,8.594837,0.0,996.0,1.0,1.0,1.0,9.0,16.0,2023.0,0.0


##Having a publisher makes any difference?

In [70]:
steam_df_indie

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
75658,Lethal Company,2023-10-23,5000000 - 10000000,169110,0,9.99,0,You are a contracted worker for the Company. Y...,"{""'English'""}",['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,zeekerssgames@gmail.com,True,False,False,0,NaN,0,110177,1342,NaN,0,97709,The game has a realistic atmosphere. Player ch...,472,433,328,308,Zeekerss,Zeekerss,"Single-player,Multi-player,Co-op,Online Co-op,...","Action,Adventure,Indie,Early Access","Early Access,Online Co-Op,Horror,First-Person,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,1067,9,1,20,1,False,82.037975,False,111519,True,False,False,10,23,2023,1,1
65147,Sons Of The Forest,2023-02-23,2000000 - 5000000,405191,0,29.99,0,An entirely new experience from the makers of ...,"{"" 'Czech'"", "" 'German'"", "" 'Turkish'"", "" 'Kor...","['English', 'Turkish']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,support@endnightgames.com,True,False,False,0,NaN,0,31006,5954,NaN,28,49027,"Sons Of The Forest contains intense violence, ...",312,312,238,238,Endnight Games Ltd,Newnight,"Single-player,Multi-player,Co-op,Online Co-op,...","Action,Adventure,Indie,Simulation,Early Access","Survival Horror,Multiplayer,Open World,Surviva...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,1447,11,4,20,16,True,5.206717,False,36960,True,False,False,2,23,2023,1,1
75414,Love Is All Around,2023-10-17,1000000 - 2000000,3368,0,9.99,0,'Love Is All Around' is a simulating love stor...,"{""'Simplified Chinese'"", "" 'Traditional Chines...","['Simplified Chinese', 'Traditional Chinese']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,intiny@jedi-games.com,True,True,True,0,NaN,0,34626,1678,NaN,42,28576,Game contains sensitive contents such as parti...,234,102,233,82,intiny,intiny,"Single-player,Multi-player,Steam Achievements,...","Adventure,Indie,RPG,Simulation,Strategy","Sci-fi,Dating Sim,Singleplayer,Visual Novel,FM...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,5487,6,1,20,3,False,20.622990,False,36304,True,False,False,10,17,2023,1,1
73026,ARK: Survival Ascended,2023-10-25,1000000 - 2000000,39814,0,44.99,0,Respawn into a new dinosaur survival experienc...,"{"" 'German'"", "" 'Turkish'"", "" 'Korean'"", "" 'Du...","['English', 'Turkish']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.playark.com,https://support.survivetheark.com/hc/en-us,support@studiowildcard.com,True,False,False,0,NaN,0,20354,14079,NaN,32,32263,NaN,3308,1170,3574,925,"Studio Wildcard,Grove Street Games,Instinct Games","Studio Wildcard,Snail Games USA","Single-player,Multi-player,MMO,PvP,Online PvP,...","Action,Adventure,Indie,Massively Multiplayer,R...","Early Access,Survival,Open World,Dinosaurs,Adv...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,3455,14,1,20,16,True,1.445597,False,34433,True,True,True,10,25,2023,1,1
73653,Party Animals,2023-09-20,2000000 - 5000000,2701,0,19.99,1,Make Painfully Great Memories with Your Friend...,"{"" 'Simplified Chinese'"", "" 'Traditional Chine...","['English', 'Simplified Chinese', 'Traditional...",NaN,https://cdn.akamai.steamstatic.com/ste

In [91]:
steam_df_indie.groupby(by="Publishers").mean(numeric_only=True)

,Peak CCU,Required age,Price,DLC count,Windows,Mac,Linux,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,about_length,n_screens,n_movies,n_tags,n_languages,has_publisher,perceived_quality,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
Publishers,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
(Myco),1307.0,0.0,6.29,0.0,1.0,0.0,0.0,0.0,0.0,267.0,5.0,NaN,17.0,334.0,114.0,114.0,114.0,114.0,1499.0,10.0,1.0,20.0,1.0,1.0,44.500000,0.0,272.0,1.0,0.0,1.0,12.0,14.0,2023.0,0.0,0.0
-乌鸦男-,1.0,0.0,0.99,0.0,1.0,0.0,0.0,0.0,0.0,108.0,10.0,NaN,0.0,116.0,0.0,0.0,0.0,0.0,424.0,6.0,1.0,20.0,1.0,0.0,9.818182,0.0,118.0,1.0,1.0,0.0,8.0,8.0,2023.0,0.0,0.0
001,2.0,0.0,1.29,0.0,1.0,0.0,0.0,0.0,0.0,36.5,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1337.5,7.0,1.0,20.0,3.0,0.0,13.700000,0.0,38.5,1.0,1.0,1.0,4.5,6.5,2023.0,0.0,0.0
0UP GAMES,205.0,0.0,15.29,0.0,1.0,0.0,0.0,0.0,0.0,348.0,2.0,NaN,14.0,371.0,0.0,0.0,0.0,0.0,1133.0,20.0,1.0,20.0,2.0,1.0,116.000000,0.0,350.0,1.0,0.0,0.0,2.0,20.0,2023.0,0.0,0.0
"100 GAMES,Gamersky Games",0.0,0.0,10.49,0.0,1.0,0.0,0.0,0.0,0.0,54.0,15.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1195.0,10.0,1.0,12.0,15.0,1.0,3.375000,0.0,69.0,1.0,0.0,0.0,3.0,17.0,2023.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
野生建模师,0.0,0.0,0.99,0.0,1.0,0.0,0.0,0.0,0.0,31.0,36.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,326.0,5.0,1.0,20.0,8.0,0.0,0.837838,0.0,67.0,1.0,0.0,0.0,8.0,29.0,2023.0,0.0,0.0
"阿地,Miradil—DHU",2.0,0.0,6.99,0.0,1.0,0.0,0.0,0.0,0.0,33.0,21.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2465.0,12.0,2.0,10.0,1.0,0.0,1.500000,0.0,54.0,1.0,1.0,0.0,4.0,11.0,2023.0,0.0,0.0
阿基米德 尼古拉斯 夏,0.0,0.0,2.39,0.0,1.0,0.0,0.0,0.0,0.0,21.0,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,750.0,9.0,1.0,19.0,2.0,1.0,2.333333,0.0,29.0,1.0,1.0,0.0,1.0,26.0,2023.0,0.0,0.0


In [87]:
list(numeric_columns).append("Publisher")
# s

In [88]:
numeric_columns

Index(['Peak CCU', 'Required age', 'Price', 'DLC count', 'Windows', 'Mac',
       'Linux', 'Metacritic score', 'User score', 'Positive', 'Negative',
       'Score rank', 'Achievements', 'Recommendations',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'about_length',
       'n_screens', 'n_movies', 'n_tags', 'n_languages', 'has_publisher',
       'perceived_quality', 'no_user_reactions', 'total_reviews',
       'has_support_email', 'has_support_url', 'has_website', 'month', 'day',
       'year', 'success_1000_threshold', 'success_500_threshold'],
      dtype='object')

In [95]:
sys.path.append("/Users/matheus/Projects/steam_analysis/steam-analysis/src/features")

In [5]:
from src.features.build_features import SteamDataTransformer

In [6]:
transformer = SteamDataTransformer()

In [7]:
transformer.fit(steam_df)

SteamDataTransformer()

In [8]:
teste = transformer.transform(steam_df)

In [9]:
teste

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
0,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,"{{""'English'""}}",[],...,False,17,False,False,True,10,21,2008,0,0
1,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{ ""'English'"", "" 'Italian'""}, "" 'Portuguese ...",[],...,False,58,True,False,True,10,12,2017,0,0
2,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"{ ""'English'""}, {"" 'Portuguese - Brazil'""}",[],...,True,0,True,False,False,11,17,2021,0,0
3,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{ "" 'Portuguese'"", "" 'Korean'"", ""'English'"",...",[],...,False,3,True,True,True,7,23,2020,0,0
4,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{ ""'English'""}, {"" 'Spanish - Spain'""}",[],...,False,58,True,True,True,2,3,2020,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78585,Mannerheim's Saloon Car,2024-01-02,0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"{ ""'English'""}, {"" 'Finnish'""}",['Finnish'],...,True,0,True,False,False,1,2,2024,0,0
78586,Beer Run,2024-01-03,0 - 0,0,0,0.00,0,Beer Run is an Indie game created to steal bee...,"{{""'English'""}}",[],...,True,0,True,False,False,1,3,2024,0,0
78587,My Friend The Spider,2024-01-04,0 - 0,0,0,0.00,0,A small 'horror' narrative game about isolatio...,"{{""'English'""}}",['English'],...,True,0,False,True,False,1,4,2024,0,0
78588,Path of Survivors,2024-01-08,0 - 0,0,0,3.99,0,Path of Survivors is a multi-class auto-battle...,"{{""'English'""}}",[],...,True,0,True,True,True,1,8,2024,0,0


In [21]:
X_transformed = steam_df.copy()
X_transformed = X_transformed.dropna(subset=["Screenshots"])
X_transformed = X_transformed.dropna(subset=["Movies"])
X_transformed = X_transformed.dropna(subset=["Genres"])
X_transformed = X_transformed.dropna(subset=["Genres"])
X_transformed = X_transformed["Tags"].fillna("empty")
X_transformed = X_transformed["About the game"].fillna("empty")
X_transformed = X_transformed.drop(columns="AppID")

KeyError: 'About the game'

In [10]:
steam_df.dropna(subset=["Screenshots"])

,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,no_user_reactions,total_reviews,has_support_email,has_support_url,has_website,month,day,year,success_1000_threshold,success_500_threshold
0,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,"{""'English'""}",[],...,False,17,False,False,True,10,21,2008,0,0
1,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"{"" 'German'"", "" 'Russian'"", "" 'Japanese'"", "" '...",[],...,False,58,True,False,True,10,12,2017,0,0
2,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"{"" 'Portuguese - Brazil'"", ""'English'""}",[],...,True,0,True,False,False,11,17,2021,0,0
3,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"{"" 'German'"", "" 'Korean'"", "" 'Portuguese'"", "" ...",[],...,False,3,True,True,True,7,23,2020,0,0
4,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"{"" 'Spanish - Spain'"", ""'English'""}",[],...,False,58,True,True,True,2,3,2020,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78585,Mannerheim's Saloon Car,2024-01-02,0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"{"" 'Finnish'"", ""'English'""}",['Finnish'],...,True,0,True,False,False,1,2,2024,0,0
78586,Beer Run,2024-01-03,0 - 0,0,0,0.00,0,Beer Run is an Indie game created to steal bee...,"{""'English'""}",[],...,True,0,True,False,False,1,3,2024,0,0
78587,My Friend The Spider,2024-01-04,0 - 0,0,0,0.00,0,A small 'horror' narrative game about isolatio...,"{""'English'""}",['English'],...,True,0,False,True,False,1,4,2024,0,0
78588,Path of Survivors,2024-01-08,0 - 0,0,0,3.99,0,Path of Survivors is a multi-class auto-battle...,"{""'English'""}",[],...,True,0,True,True,True,1,8,2024,0,0
